In [16]:
import os
from dotenv import load_dotenv

# LangChain 관련 모듈 가져오기
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

# .env 파일 로드
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 1. Pydantic을 사용한 학생 정보 데이터 모델 정의
class StudentInfo(BaseModel):
    """학생의 이름, 나이, 전공, 취미, 목표 정보를 추출합니다."""
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 목록")
    goal: str = Field(description="학생의 장래 목표")

# 2. PydanticOutputParser 초기화
# Pydantic 모델을 파서에 연결하여 모델의 출력 구조를 정의
parser = PydanticOutputParser(pydantic_object=StudentInfo)

# 3. 모델 초기화
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  
    model="meta-llama/llama-4-scout-17b-16e-instruct",  
    temperature=0.7
)

# 4. 프롬프트 템플릿 설정
# 자유 형식의 텍스트에서 구조화된 정보를 추출하도록 지시
# {format_instructions}에 파서가 요구하는 JSON 스키마가 자동으로 삽입됨
prompt = PromptTemplate(
    template="""당신은 주어진 텍스트에서 학생의 정보를 구조화하는 전문가입니다.
주어진 텍스트에서 학생의 이름, 나이, 전공, 취미, 목표를 추출하여 JSON 형식으로 반환하세요.

{format_instructions}

자유 형식 자기소개:
{text}""",
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 5. LLM Chain 구성
# 프롬프트, 모델, 파서를 연결하는 파이프라인
chain = prompt | llm | parser

# 6. 테스트 입력
input_text = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. " \
"취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

# 7. 실행 및 결과 출력
try:
    # 체인 실행
    student_info = chain.invoke({"text": input_text})

    # Pydantic 모델 객체의 내용을 깔끔하게 출력
    print("--- 학생 정보 구조화 결과 ---")
    print(student_info.model_dump_json(indent=4))

except Exception as e:
    print(f"정보 추출 실패: {e}")


--- 학생 정보 구조화 결과 ---
{
    "name": "김민수",
    "age": 22,
    "major": "컴퓨터공학",
    "hobbies": [
        "게임하기",
        "영화보기",
        "코딩"
    ],
    "goal": "훌륭한 개발자가 되는 것"
}
